In [ ]:
from global_parameters import GlobalParameters
from states import Key, Ciphertext
from decryption_circuit import decrypt_results
from verification_circuit import verify_deletion_counts
from utils import import_counts
from qiskit.circuit import qpy_serialization
import shutil

In [ ]:
# Unzip files
shutil.unpack_archive("data_export.zip", "data")

In [ ]:
# Import all files
with open("data/global_params.txt", "r") as params_file, open("data/key.txt", "r") as key_file, open("data/ciphertext.txt", "r") as ciphertext_file, open("data/message.txt", "r") as message_file:
    params = GlobalParameters.from_json(params_file.read())
    key = Key.from_json(key_file.read())
    ciphertext = Ciphertext.from_json(ciphertext_file.read(), "data/base_circuit.qpy")
    message = message_file.read()

In [ ]:
# Test 1 - delete
deletion_counts_test1 = import_counts("data/test1-deletion-counts.csv")
verify_deletion_counts(deletion_counts_test1, key, params);

In [ ]:
# Test 2 - decrypt
decryption_counts_test2 = import_counts("data/test2-decryption-counts.csv")
decrypt_results(decryption_counts_test2, key, ciphertext, message)

In [ ]:
# Test 3 - delete (possibly maliciously) and then decrypt
raw_counts_test3 = import_counts("data/test3-raw-counts.csv")
deletion_counts_test3 = import_counts("data/test3-deletion-counts.csv")
decryption_counts_test3 = import_counts("data/test3-decryption-counts.csv")
accepted_certificates_test3 = verify_deletion_counts(deletion_counts_test3, key, params)
# Test the decryption results for all the measurements, regardless of whether or not the certificate of deletion was accepted
decrypt_results(decryption_counts_test3, key, ciphertext, message)

# Test the decryption results for just the measurements where the certificate of deletion was accepted
accepted_deletion_decryption_counts_test3 = {}
for measurement, count in raw_counts_test3.items():
    deletion_measurement, decryption_measurement = measurement.split(" ")
    if deletion_measurement in accepted_certificates_test3:
        accepted_deletion_decryption_counts_test3[decryption_measurement] = accepted_deletion_decryption_counts_test3.get(decryption_measurement, 0) + count

print()
print("Of the measurements where the proof of deletion was accepted, the following are the decryption statistics:")
decrypt_results(accepted_deletion_decryption_counts_test3, key, ciphertext, message)